# Redes Neurais Recorrentes

Antes de iniciarmos com as redes neurais recorrentes, é melhor se acostumar com algumas operações comuns do PyTorch. Vamos iniciar com as suas unidades mais básicas:tensores. Suas principais operações aritméticas, formas de inicialização, tensores "atualizáveis" entre outros. Em seguida vamos construir uma simples regressão linear usando apenas tensores. A seguir, construímos uma rede neural usando o módulo `nn` do framework, vendo que a sua construção é exatamente a mesma usada pela rede neural de tensores. Finalmente, vamos entender como criar redes neurais customizáveis por herança da classe `Module` (um dos piores nomes de classe que já vi na vida...).

In [1]:
import torch
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
from context import fakenews
from fakenews import preprocess as pre
import gensim
import math

### Tensores

É a principal estrutura de dado do framework. Ela é essencialmente um array numpy de 3 dimensões. Entretanto, com tensores é possível realizar operações diretamente na GPU aumentando a eficiência dos algoritmos. Além disso, eles permitem que o gradiente seja calculado de forma automática a partir de um parâmetro.

In [2]:
torch.rand(2, 3, 2)

tensor([[[0.9979, 0.5295],
         [0.0180, 0.3366],
         [0.3373, 0.7002]],

        [[0.8437, 0.1855],
         [0.4123, 0.9194],
         [0.4782, 0.7464]]])

Temos suporte direto a operações aritméticas assim como no numPy. Podemos realizar adições (`+`), subtrações (`-`), multiplicação por escalar (`*`), transposição (`T` ou `transp()`), produto de hadarmard (`*`), produto interno (`@`), entre outros.

In [3]:
A = torch.ones(2, 3, 2, dtype=torch.int) * 2
I = torch.eye(3, 2, dtype=torch.int)

In [4]:
A

tensor([[[2, 2],
         [2, 2],
         [2, 2]],

        [[2, 2],
         [2, 2],
         [2, 2]]], dtype=torch.int32)

In [5]:
I

tensor([[1, 0],
        [0, 1],
        [0, 0]], dtype=torch.int32)

In [6]:
A + torch.ones(2, 3, 2, dtype=torch.int)

tensor([[[3, 3],
         [3, 3],
         [3, 3]],

        [[3, 3],
         [3, 3],
         [3, 3]]], dtype=torch.int32)

In [7]:
A * I

tensor([[[2, 0],
         [0, 2],
         [0, 0]],

        [[2, 0],
         [0, 2],
         [0, 0]]], dtype=torch.int32)

In [8]:
# A * I == A @ I

In [9]:
A @ torch.eye(2, dtype=torch.int)

tensor([[[2, 2],
         [2, 2],
         [2, 2]],

        [[2, 2],
         [2, 2],
         [2, 2]]], dtype=torch.int32)

In [10]:
np.pi * A

tensor([[[6.2832, 6.2832],
         [6.2832, 6.2832],
         [6.2832, 6.2832]],

        [[6.2832, 6.2832],
         [6.2832, 6.2832],
         [6.2832, 6.2832]]])

In [11]:
rng = torch.rand(1, 3, 2)
A + rng

tensor([[[2.0242, 2.7475],
         [2.2647, 2.4919],
         [2.3866, 2.0739]],

        [[2.0242, 2.7475],
         [2.2647, 2.4919],
         [2.3866, 2.0739]]])

In [12]:
A.view(-1, 4)

tensor([[2, 2, 2, 2],
        [2, 2, 2, 2],
        [2, 2, 2, 2]], dtype=torch.int32)

A execução na GPU não ocorre de forma automática. Primeiro, podemos verificar se o dispositivo é suportado pelo PyTorch.

In [13]:
torch.cuda.is_available()

/home/thalesaguiar/.pyenv/versions/3.8.2/envs/fakenews.env/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


False

Em caso positivo, podemos especificar quais tensores terão suas operações executadas na placa gráfica, definir a execução de tudo por padrão.

In [14]:
# gpu = torch.device("cuda")
# A.to(gpu)
# cuda_tensor = torch.tensor([3, 4, 5], device=gpu)

Algumas operações do python geram uma **view** da coleção. Um exemplo comum são os slices. Essa é uma operação bastante comum no NumPy e praticamente todos os frameworks baseados nele. Entretanto, é importante notar que isso gera uma espécie de **ponteiro** para o vetor original. Isto é, alterações na view são alterações no vetor original, pois a view é nada mais que uma visualização diferente do mesmo espaço de memória.

In [15]:
a_view = A.view(-1, 3)
a_view

tensor([[2, 2, 2],
        [2, 2, 2],
        [2, 2, 2],
        [2, 2, 2]], dtype=torch.int32)

In [16]:
A[0, 0, 0] = 99999
a_view

tensor([[99999,     2,     2],
        [    2,     2,     2],
        [    2,     2,     2],
        [    2,     2,     2]], dtype=torch.int32)

In [17]:
a_view[0, 2] = 55
A

tensor([[[99999,     2],
         [   55,     2],
         [    2,     2]],

        [[    2,     2],
         [    2,     2],
         [    2,     2]]], dtype=torch.int32)

para realizar uma cópia, usamos o operador `.clone()`. Assim, é criada uma nova estrutura onde cada elemento é uma cópia dos valores originais, sem preservar qualquer referência.

In [18]:
Acp = A.clone()
Acp[0, 2, 1] = 33
(A[0], Acp[0])

(tensor([[99999,     2],
         [   55,     2],
         [    2,     2]], dtype=torch.int32),
 tensor([[99999,     2],
         [   55,     2],
         [    2,    33]], dtype=torch.int32))

## Rede Neural

O PyTorch permite criar modelos neurais com bastante facilidade e praticidade. Tanto a criação de modelos customizáveis e a grande quantidade de modelos prontos disponíveis é bastante simples. Vamos gerar um conjunto de dados aleatórios para experimentar as funções do framework.

In [19]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

X, Y = load_boston(return_X_y=True)
norm = StandardScaler()
Xstd = norm.fit_transform(X)
Xtrain, Xtest, Ytrain, Ytest = train_test_split(Xstd, Y, test_size=0.3,
                                                shuffle=True)
Xdf = pd.DataFrame(Xstd)
Xdf.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,-0.419782,0.284830,-1.287909,-0.272599,-0.144217,0.413672,-0.120013,0.140214,-0.982843,-0.666608,-1.459000,0.441052,-1.075562
1,-0.417339,-0.487722,-0.593381,-0.272599,-0.740262,0.194274,0.367166,0.557160,-0.867883,-0.987329,-0.303094,0.441052,-0.492439
2,-0.417342,-0.487722,-0.593381,-0.272599,-0.740262,1.282714,-0.265812,0.557160,-0.867883,-0.987329,-0.303094,0.396427,-1.208727
3,-0.416750,-0.487722,-1.306878,-0.272599,-0.835284,1.016303,-0.809889,1.077737,-0.752922,-1.106115,0.113032,0.416163,-1.361517
4,-0.412482,-0.487722,-1.306878,-0.272599,-0.835284,1.228577,-0.511180,1.077737,-0.752922,-1.106115,0.113032,0.441052,-1.026501


A base carregada é a Boston-Housing prices, uma das disponíveis diretamente no SkLearn. Agora precisamos transformar os conjuntos para usá-los no PyTorch, ou seja, precisamos criar tensores.

In [20]:
Xtrain = torch.from_numpy(Xtrain).type(torch.float)
Xtest = torch.from_numpy(Xtest).type(torch.float)
Ytrain = torch.from_numpy(Ytrain).type(torch.float).flatten()
Ytest = torch.from_numpy(Ytest).type(torch.float).flatten()
(Xtrain[:2], Ytrain[:2])

(tensor([[-0.4169,  3.5896, -1.4105, -0.2726, -1.3104,  0.9835, -1.8945,  1.8341,
          -0.7529, -0.0370, -0.6730,  0.4411, -1.1344],
         [-0.3904, -0.4877,  1.5690, -0.2726,  0.5987, -0.8429,  0.9753, -0.9539,
          -0.6380,  0.1708,  1.2689,  0.3885,  0.6360]]),
 tensor([34.9000, 16.2000]))

Com os dados devidamente ajustados, vamos criar um modelo de regressão linear usando apenas tensores. Primeiro, definimos os `weights` e em seguida o `bias`.

In [21]:
weights = torch.rand(1, 13, requires_grad=True)
weights

tensor([[0.8673, 0.4359, 0.7215, 0.7371, 0.4819, 0.1158, 0.3375, 0.2009, 0.9142,
         0.2180, 0.3907, 0.3067, 0.3894]], requires_grad=True)

In [22]:
bias = torch.rand(1, 1, requires_grad=True)
bias

tensor([[0.6014]], requires_grad=True)

O parâmetro `requires_grad` diz ao PyTorch que esse tensor deve ser levado em consideração quando algum otimizador for utilizado. Ou seja, no momento que fazemos a propagação do erro, esse tensor será atualizado.

Então, temos o seguinte problema:

$$y = D \times W + b$$

Onde $D$ é nossa base (apenas *features*), $W$ são os pesos, $b$ é um bias representando possíveis ruídos/erros obtidos das aproximações e $y$ são nossos valores esperados. Ou seja

$$\hat{y} = D \times W^{'} + b^{'}$$

Sendo $\hat{y}$ um valor próximo o suficiente de $y$, assim como $W^{'}$ e $b^{'}$.

In [23]:
yhat = Xtrain @ weights.T + bias
yhat[:3]

tensor([[-1.4681],
        [ 1.6262],
        [ 5.0308]], grad_fn=<SliceBackward>)

In [24]:
def model(X):
    return X @ weights.T + bias

Agora, precisamos definir a função objetivo. Vamos usar um simples Mean Squared Error.

In [25]:
sqr_diff = (Ytrain - yhat) ** 2
torch.sum(sqr_diff) / sqr_diff.numel()

tensor(565.2597, grad_fn=<DivBackward0>)

A operação `numel()` retorna a quantidade de elementos do tensor. Vamos criar uma função para calcular o erro

In [26]:
def mse(preds, real):
    diff = (real - preds) ** 2
    return torch.sum(diff) / diff.numel()

Para descobrir o quanto atualizar para cada peso, vamos aplicar o *gradient descent*. 

In [27]:
loss = mse(Ytrain, yhat)
loss.backward()
weights

tensor([[0.8673, 0.4359, 0.7215, 0.7371, 0.4819, 0.1158, 0.3375, 0.2009, 0.9142,
         0.2180, 0.3907, 0.3067, 0.3894]], requires_grad=True)

Note que os valores do tensor não foram atualizados após executar o backpropagation. Isso é porque os valores do gradiente são armazenados no atributo `grad` de cada tensor

In [28]:
weights.grad

tensor([[ 5.7873, -0.3379,  3.3922,  3.1683,  3.1507, -1.2797,  1.6849, -2.2740,
          5.9151,  5.1550,  2.4426, -3.8842,  2.0310]])

Agora, podemos atualizar o tensor com o seu gradiente. Aqui devemos tomar cuidado, pois o PyTorch controla e memoriza todas as operações realizadas nos tensores com `require_grad=True` para utilizar no cálculo dos gradientes. Ao atualizar os parâmetros, não queremos que essa operação seja gravada. Para evitar esse comortamento, podemos usar o operador de contexto `with` com a função `torch.no_grad()`.

Esse comportamento é devido a estratégia para automaticamente gerar o gradiente de qualquer estrutura. O PyTorch (e provavelmente outros) transforma o gradiente descendente/backpropagation em um grafo onde cada nó é uma operação. Assim, ele só precisa gerar a derivada de cada nó e aplicar o backpropagation.

In [29]:
# hyperparameters
weights.grad.zero_()
bias.grad.zero_()

nepochs = 500
lrate = 1e-2

for epoch in range(nepochs):
    predictions = model(Xtrain)
    loss = mse(predictions, Ytrain)
    loss.backward()
    with torch.no_grad():
        weights -= lrate * weights.grad
        bias -= lrate * bias.grad
        weights.grad.zero_()
        bias.grad.zero_()
    if epoch % 25 == 0:
        print(f'{epoch:3}° {loss}')

  0° 565.2597045898438
 25° 253.29234313964844
 50° 143.78929138183594
 75° 104.25788116455078
100° 89.80180358886719
125° 84.44068145751953
150° 82.41802215576172
175° 81.63721466064453
200° 81.32584381103516
225° 81.19577026367188
250° 81.13782501220703
275° 81.10980224609375
300° 81.09490203857422
325° 81.086181640625
350° 81.08063507080078
375° 81.07685089111328
400° 81.07413482666016
425° 81.07209777832031
450° 81.07054138183594
475° 81.06930541992188


In [30]:
pred = model(Xtest)
loss = mse(pred, Ytest)
loss

tensor(92.3383, grad_fn=<DivBackward0>)

### Otimizador

Também podemos selecionar qual tipo de otimizador usaremos para ajustar os parâmetros. O mais comum é o Stochastic Gradient Descent (SGD), mas o PyTorch oferece o Adadelta, Adagrad, RMSProp, entre vários outros dentro do pacote `torch.optim`. Além disso, claro, também podemos customizar e criar o nosso próprio otimizador. Mas essa customização não está dentro do nosso escopo.

In [31]:
?torch.optim.RMSprop

Init signature:
torch.optim.RMSprop(
    params,
    lr=0.01,
    alpha=0.99,
    eps=1e-08,
    weight_decay=0,
    momentum=0,
    centered=False,
)
Docstring:     
Implements RMSprop algorithm.

Proposed by G. Hinton in his
`course <https://www.cs.toronto.edu/~tijmen/csc321/slides/lecture_slides_lec6.pdf>`_.

The centered version first appears in `Generating Sequences
With Recurrent Neural Networks <https://arxiv.org/pdf/1308.0850v5.pdf>`_.

The implementation here takes the square root of the gradient average before
adding epsilon (note that TensorFlow interchanges these two operations). The effective
learning rate is thus :math:`\alpha/(\sqrt{v} + \epsilon)` where :math:`\alpha`
is the scheduled learning rate and :math:`v` is the weighted moving average
of the squared gradient.

Arguments:
    params (iterable): iterable of parameters to optimize or dicts defining
        parameter groups
    lr (float, optional): learning rate (default: 1e-2)
    momentum (float, optional): momen

Em geral, os otimizadores são variações do Gradient Descent/Backpropagation. Mas alguns são melhores para evitar cair em mínimos locais, ou mesmo para reduzir a quantidade de épocas necessárias. Outros, como o Adam, podem aumentar a complexidade da otimização ao utilizar valores diferentes para cada parâmetro, consequentemente aumentando também a flexibilidade.

In [32]:
network = torch.nn.Linear(13, 1)
print(network.weight)
print(network.bias)

Parameter containing:
tensor([[ 0.0897, -0.1303,  0.0116, -0.2762, -0.0097, -0.1162,  0.2734, -0.1749,
         -0.1626,  0.0609, -0.0571, -0.0592, -0.1353]], requires_grad=True)
Parameter containing:
tensor([0.2699], requires_grad=True)


A classe acima é uma **layer** linear. Como podemos ver, a sua implementação segue a mesma estrutura que usamos na nossa regressão linear. Usando os pesos e bias como parâmetros (tensores) e o autograd para atualização deles.

Outro módulo bastante usado do framework é o `functional`. Ele possui várias implementações de funções, tanto para ativação quanto para definição de funções objetivo. Abaixo, definimos noss função objetivo novamente como a o MSE, sendo que dessa vez usando a implementação do próprio PyTorch.

In [33]:
mse = torch.nn.functional.mse_loss
?mse

Signature: mse(input, target, size_average=None, reduce=None, reduction='mean')
Docstring:
mse_loss(input, target, size_average=None, reduce=None, reduction='mean') -> Tensor

Measures the element-wise mean squared error.

See :class:`~torch.nn.MSELoss` for details.
File:      ~/.pyenv/versions/3.8.2/envs/fakenews.env/lib/python3.8/site-packages/torch/nn/functional.py
Type:      function


Logo em seguida, o módulo `optim` encapsula uma grande variedade de otimizadores. Abaixo, definimos o mais padrão (SGD) e o Adam.

In [34]:
sgd = torch.optim.SGD(network.parameters(), lr=1e-5)
adam = torch.optim.Adam(network.parameters(), lr=1e-5)

## DataLoader

PyTorch permite encapsular a base de dados para melhor manipular o treinamento e batches da forma mais "pythonica" possível.

In [35]:
tensordata = torch.utils.data.TensorDataset(Xtrain, Ytrain)
tensordata[:2]

(tensor([[-0.4169,  3.5896, -1.4105, -0.2726, -1.3104,  0.9835, -1.8945,  1.8341,
          -0.7529, -0.0370, -0.6730,  0.4411, -1.1344],
         [-0.3904, -0.4877,  1.5690, -0.2726,  0.5987, -0.8429,  0.9753, -0.9539,
          -0.6380,  0.1708,  1.2689,  0.3885,  0.6360]]),
 tensor([34.9000, 16.2000]))

In [36]:
batch_size = 25
dtl = torch.utils.data.DataLoader(tensordata, batch_size, shuffle=True)

A variável `dtl` é **generator** para a base de dados. Cada iteração sobre ela retorna `batch_size` amostras para treino, assim como seus repectivos valores alvo.

## Treinamento

Por fim, para realizarmos o treinamento do nosso modelo, a fórmula é bem parecida com a usada na nossa rede com apenas tensores. Na verdade, essa parte do código para treinamento acaba se tornando um certo padrão para modelos do PyTorch.

In [37]:
nepochs = 500
lrate = 1e-5

for epoch in range(nepochs):
    for Xbatch, Ybatch in dtl:
        pred = network(Xbatch)
        loss = mse(pred.flatten(), Ybatch)
        loss.backward(loss)
        adam.step()
        adam.zero_grad()
    if epoch % 25 == 0:
        print(f'{epoch:3}° {loss}')

  0° 869.1465454101562
 25° 932.4745483398438
 50° 478.1632080078125
 75° 887.9330444335938
100° 1426.0372314453125
125° 913.5899658203125
150° 647.532958984375
175° 158.18785095214844
200° 384.00115966796875
225° 798.8470458984375
250° 637.3695678710938
275° 279.03741455078125
300° 673.9016723632812
325° 439.4954528808594
350° 318.876220703125
375° 395.7188415527344
400° 381.6651306152344
425° 483.4902038574219
450° 264.4954833984375
475° 673.796630859375


Note que há uma variação no erro, na qual ele nem sempre está reduzindo. consegue identificar o motivo? O que estamos fazendo de diferente com relação ao nosso modelo com apenas tensores?

# Redes custom

Usando a classe `Module` (novamente, pior nome possível), podemos, a partir de herança, criar arquiteturas de de redes inimagináveis. Isso deve-se ao grande poder do autograd.tudo que precisamos é definir como os dados são passados em direção a saída da rede, ou seja, o *forward pass*. Tendo em vista que o autograd se encarrega de memorizar como os dados foram propagados para frente, gerando automaticamente as operações que devem ser usados durante o gradient descent.

In [45]:
class SigmoidNN(torch.nn.Module):
    
    def __init__(self, din, dh, dout):
        super().__init__()
        self.input = torch.nn.Linear(din, dh)
        self.hidden = torch.nn.Linear(dh, dout)
        self.out = torch.nn.Linear(dout, 1)
        self.activation = torch.nn.functional.sigmoid
        
    def forward(self, data):
        l1out = self.activation(self.input(data))
        l2out = self.activation(self.hidden(l1out))
        pred = self.out(l2out)
        return torch.nn.functional.tanh(pred)

Acima, definimos uma rede neural com 3 layers, sendo duas camadas sigmoid seguidas de uma camada totalmente conectada a qual realiza a regressão usando uma tangente hiperbólica.

In [44]:
model = SigmoidNN(13, 50, 10)

criterion = torch.nn.functional.mse_loss
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
for t in range(2000):
    y_pred = model(Xtrain)
    loss = criterion(y_pred.flatten(), Ytrain)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if t % 100 == 99:
        print(t, loss.item())

99 601.2274780273438
199 600.5746459960938
299 599.9262084960938
399 599.281982421875
499 598.6421508789062
599 598.0067138671875
699 597.3755493164062
799 596.748779296875
899 596.12646484375
999 595.5086059570312
1099 594.895263671875
1199 594.2864990234375
1299 593.6821899414062
1399 593.0826416015625
1499 592.4877319335938
1599 591.8973388671875
1699 591.311767578125
1799 590.7310180664062
1899 590.1550903320312
1999 589.5839233398438
